# Initialization and Optimization

For this lab on initialization and optimization, let's look at a slightly different type of neural network. This time, we will not perform a classification task as we've done before.  Instead, we'll look at a regression problem.

We can just as well use deep learning networks for regression as for a classification problem. However, note that getting regression to work with neural networks is a harder problem because the output is unbounded ($\hat y$ can technically range from $-\infty$ to $+\infty$, and the models are especially prone to **_exploding gradients_**. This issue makes a regression exercise the perfect learning case!

Run the cell below to import everything we'll need for this lab.

In [1]:
import numpy as np
np.random.seed(0)
import pandas as pd
from keras.models import Sequential
from keras import initializers
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn import preprocessing
from keras import optimizers
from sklearn.model_selection import train_test_split

/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


## 1. Loading the data

The data we'll be working with is data related to facebook posts published during the year of 2014 on the Facebook's page of a renowned cosmetics brand.  It includes 7 features known prior to post publication, and 12 features for evaluating the post impact. What we want to do is make a predictor for the number of "likes" for a post, taking into account the 7 features prior to posting.

First, let's import the data set and delete any rows with missing data.  

The dataset is contained with the file `dataset_Facebook.csv`. In the cell below, use pandas to read in the data from this file. Because of the way the data is structure, make sure you also set the `sep` parameter to `";"`, and the `header` parameter to `0`. 

Then, use the DataFrame's built-in `.dropna()` function to remove any rows with missing values. 

In [4]:
# load dataset
data = pd.read_csv('dataset_Facebook.csv', sep=';', header=0)
data = data.dropna()

Now, let's check the shape of our data to ensure that everything looks correct. 

In [5]:
np.shape(data) #Expected Output: (495, 19)

(495, 19)

And finally, let's inspect the `.head()` of the DataFrame to get a feel for what our dataset looks like. 

In [6]:
data.head()

,Page total likes,Type,Category,Post Month,Post Weekday,Post Hour,Paid,Lifetime Post Total Reach,Lifetime Post Total Impressions,Lifetime Engaged Users,Lifetime Post Consumers,Lifetime Post Consumptions,Lifetime Post Impressions by people who have liked your Page,Lifetime Post reach by people who like your Page,Lifetime People who have liked your Page and engaged with your post,comment,like,share,Total Interactions
0,139441,Photo,2,12,4,3,0.0,2752,5091,178,109,159,3078,1640,119,4,79.0,17.0,100
1,139441,Status,2,12,3,10,0.0,10460,19057,1457,1361,1674,11710,6112,1108,5,130.0,29.0,164
2,139441,Photo,3,12,3,3,0.0,2413,4373,177,113,154,2812,1503,132,0,66.0,14.0,80
3,139441,Photo,2,12,2,10,1.0,50128,87991,2211,790,1119,61027,32048,1386,58,1572.0,147.0,1777
4,139441,Photo,2,12,2,3,0.0,7244,13594,671,410,580,6228,3200,396,19,325.0,49.0,393


## 2. Initialization

### 2.1 Normalize the input data

A big part of Deep Learning is cleaning the data and getting into a shape usable by a neural network.  Let's get some additional practice with this.


Take a look at our input data. We'll use the 7 first columns as our predictors. We'll do the following two things:
- Normalize the continuous variables --> you can do this using `np.mean()` and `np.std()`
- make dummy variables of the categorical variables (you can do this by using `pd.get_dummies`)

We only count "Category" and "Type" as categorical variables. Note that you can argue that "Post month", "Post Weekday" and "Post Hour" can also be considered categories, but we'll just treat them as being continuous for now.

In the cell below, convert the data as needed by normalizing or converting to dummy variables, and then concatenate it all back into a single DataFrame once you've finished.  

In [9]:
data.head()

,Page total likes,Type,Category,Post Month,Post Weekday,Post Hour,Paid,Lifetime Post Total Reach,Lifetime Post Total Impressions,Lifetime Engaged Users,Lifetime Post Consumers,Lifetime Post Consumptions,Lifetime Post Impressions by people who have liked your Page,Lifetime Post reach by people who like your Page,Lifetime People who have liked your Page and engaged with your post,comment,like,share,Total Interactions
0,139441,Photo,2,12,4,3,0.0,2752,5091,178,109,159,3078,1640,119,4,79.0,17.0,100
1,139441,Status,2,12,3,10,0.0,10460,19057,1457,1361,1674,11710,6112,1108,5,130.0,29.0,164
2,139441,Photo,3,12,3,3,0.0,2413,4373,177,113,154,2812,1503,132,0,66.0,14.0,80
3,139441,Photo,2,12,2,10,1.0,50128,87991,2211,790,1119,61027,32048,1386,58,1572.0,147.0,1777
4,139441,Photo,2,12,2,3,0.0,7244,13594,671,410,580,6228,3200,396,19,325.0,49.0,393


In [7]:
X0 = data["Page total likes"]
X1 = data["Type"]
X2 = data["Category"]
X3 = data["Post Month"]
X4 = data["Post Weekday"]
X5 = data["Post Hour"]
X6 = data["Paid"]

## standardize/categorize
X0= (X0-np.mean(X0))/np.std(X0)
dummy_X1= pd.get_dummies(X1)
dummy_X2= pd.get_dummies(X2)
X3= (X3-np.mean(X3))/np.std(X3)
X4= (X4-np.mean(X4))/np.std(X4)
X5= (X5-np.mean(X5))/np.std(X5)

# Add them all back into a single DataFrame
X = pd.concat([X0, dummy_X1, dummy_X2, X3, X4, X5, X6], axis=1)

# Store our labels in a separate variable
Y = data["like"]

In [14]:
X

,Page total likes,Link,Photo,Status,Video,1,2,3,Post Month,Post Weekday,Post Hour,Paid
0,1.004960,0,1,0,0,0,1,0,1.506154,-0.065724,-1.105878,0.0
1,1.004960,0,0,1,0,0,1,0,1.506154,-0.558655,0.492065,0.0
2,1.004960,0,1,0,0,0,0,1,1.506154,-0.558655,-1.105878,0.0
3,1.004960,0,1,0,0,0,1,0,1.506154,-1.051585,0.492065,1.0
4,1.004960,0,1,0,0,0,1,0,1.506154,-1.051585,-1.105878,0.0
5,1.004960,0,0,1,0,0,1,0,1.506154,-1.544516,0.263787,0.0
6,1.004960,0,1,0,0,0,0,1,1.506154,-1.544516,-1.105878,1.0
7,1.004960,0,1,0,0,0,0,1,1.506154,1.413068,0.263787,1.0
8,1.004960,0,0,1,0,0,1,0,1.506154,1.413068,-1.105878,0.0
9,1.004960,0,1,0,0,0,0,1,1.506154,0.920137,0.492065,0.0


In [ ]:
#Note: you get the same result for standardization if you use StandardScaler from sklearn.preprocessing

#from sklearn.preprocessing import StandardScaler
#sc = StandardScaler()
#X0 = sc.fit_transform(X0)

Our data is fairly small. Let's just split the data up in a training set and a validation set!

In the cell below:

* Split the data into training and testing sets by passing `X` and `Y` into `train_test_split`.  Set a `test_size` of `0.2`.

In [15]:
X_train, X_val, Y_train, Y_val = train_test_split(X,Y, test_size = .2)

Let's check the shape to make sure everything worked correctly.

In [16]:
X_val.shape # Expected Output: (99, 12)

(99, 12)

In [17]:
X_train.shape # Expected Output: (396, 12)

(396, 12)

## Building a Neural Network for Regression

Now, we'll build a neural network to predict the number of likes we think a post will receive.  

In the cell below, create a model with the following specifications:

* 1 Hidden Layer with 8 neurons.  In this layer, also set `input_dim` to `12`, and `activation` to `"relu"`.
* An output layer with 1 neuron.  For this neuron, set the activation to `linear`.  

In [18]:
model = Sequential()
model.add(Dense(8,activation='relu',input_dim = 12))
model.add(Dense(1,activation='linear'))

Now, we need to compile the model, with the following hyperparameters:

* `optimizer='sgd'`
* `loss='mse'`
* `metrics=['mse']`

Note that since our model is training for a regression task, not a classification task, we'll need to use a loss metric that corresponds with regression tasks--Mean Squared Error. 

In [19]:
model.compile(optimizer='sgd', loss='mse', metrics=['mse'])

Finally, let's train the model.  Call `model.fit()`. In addition to to the training data and labels, also set:

* `batch_size=32`
* `epochs=100`
* `verbose=1`
* `validation_data=(X_val, y_val)`

In [21]:
model.fit(X_train, Y_train, batch_size=32,epochs=100,verbose=1, validation_data=(X_val, Y_val))

Train on 396 samples, validate on 99 samples
Epoch 1/100
396/396 [==============================] - 0s 510us/step - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 2/100
396/396 [==============================] - 0s 69us/step - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 3/100
396/396 [==============================] - 0s 55us/step - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 4/100
396/396 [==============================] - 0s 68us/step - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 5/100
396/396 [==============================] - 0s 52us/step - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 6/100
396/396 [==============================] - 0s 63us/step - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 7/100
396/396 [=====================

396/396 [==============================] - 0s 72us/step - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 55/100
396/396 [==============================] - 0s 52us/step - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 56/100
396/396 [==============================] - 0s 40us/step - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 57/100
396/396 [==============================] - 0s 38us/step - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 58/100
396/396 [==============================] - 0s 39us/step - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 59/100
396/396 [==============================] - 0s 38us/step - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 60/100
396/396 [==============================] - 0s 51us/step - loss: nan - mean_squared

Did you see what happend? all the values for training and validation loss are "nan". There could be several reasons for that, but as we already mentioned there is likely a vanishing or exploding gradient problem.  This means that the values got so large or so small that they no longer fit in memory.   R

Recall that we normalized out inputs. But how about the outputs? Let's have a look.

In [22]:
Y_train.head()

464    268.0
386     48.0
144     41.0
167    485.0
29     143.0
Name: like, dtype: float64

Yes, indeed. We didn't normalize them and we should, as they take pretty high values. Let
s rerun the model but make sure that the output is normalized as well!

### 2.2 Normalizing the output

In the cell below, we've included all the normalization code that we wrote up top, but this time, we've added a line to normalize the data in `Y`, as well. This should help alot!

Run the cell below to normalize our data and our labels.

In [55]:
X0 = data["Page total likes"]
X1 = data["Type"]
X2 = data["Category"]
X3 = data["Post Month"]
X4 = data["Post Weekday"]
X5 = data["Post Hour"]
X6 = data["Paid"]

## standardize/categorize
X0= (X0-np.mean(X0))/(np.std(X0))
dummy_X1= pd.get_dummies(X1)
dummy_X2= pd.get_dummies(X2)
X3= (X3-np.mean(X3))/(np.std(X3))
X4= (X4-np.mean(X4))/(np.std(X4))
X5= (X5-np.mean(X5))/(np.std(X5))

X = pd.concat([X0, dummy_X1, dummy_X2, X3, X4, X5, X6], axis=1)

Y = (data["like"]-np.mean(data["like"]))/(np.std(data["like"]))

Now, let's split our data into appropriate training and testing sets again.  Split the data, just like we did before.  Use the same `test_size` as we did last time, too. 

In [56]:
X_train, X_val, Y_train, Y_val = train_test_split(X, Y, test_size=0.2)

Now, let's reinitialize our model and build it from scratch again.  

**_NOTE:_**  If we don't reinitialize our model, our training would start with the weight values we ended with during the last training session.  In order to start fresh, we need to declare a new `Sequential()` object.  

Build the model with the exact same architecture and hyperparameters as we did above.

In [57]:
model = Sequential()
model.add(Dense(8,activation='relu',input_dim = 12))
model.add(Dense(1,activation='linear'))

Now, compile the model with the same parameters we used before.

In [58]:
model.compile(optimizer='sgd', loss='mse', metrics=['mse'])

And finally, fit the model using the same parameters we did before. 

In [59]:
hist = model.fit(X_train, Y_train, batch_size=32,epochs=100,verbose=1, validation_data=(X_val, Y_val))

Train on 396 samples, validate on 99 samples
Epoch 1/100
396/396 [==============================] - 0s 1ms/step - loss: 1.4566 - mean_squared_error: 1.4566 - val_loss: 0.3553 - val_mean_squared_error: 0.3553
Epoch 2/100
396/396 [==============================] - 0s 47us/step - loss: 1.3208 - mean_squared_error: 1.3208 - val_loss: 0.3169 - val_mean_squared_error: 0.3169
Epoch 3/100
396/396 [==============================] - 0s 46us/step - loss: 1.2740 - mean_squared_error: 1.2740 - val_loss: 0.3081 - val_mean_squared_error: 0.3081
Epoch 4/100
396/396 [==============================] - 0s 68us/step - loss: 1.2513 - mean_squared_error: 1.2513 - val_loss: 0.3015 - val_mean_squared_error: 0.3015
Epoch 5/100
396/396 [==============================] - 0s 57us/step - loss: 1.2360 - mean_squared_error: 1.2360 - val_loss: 0.3075 - val_mean_squared_error: 0.3075
Epoch 6/100
396/396 [==============================] - 0s 66us/step - loss: 1.2265 - mean_squared_error: 1.2265 - val_loss: 0.2944 - val

396/396 [==============================] - 0s 60us/step - loss: 1.1133 - mean_squared_error: 1.1133 - val_loss: 0.2787 - val_mean_squared_error: 0.2787
Epoch 51/100
396/396 [==============================] - 0s 58us/step - loss: 1.1114 - mean_squared_error: 1.1114 - val_loss: 0.2798 - val_mean_squared_error: 0.2798
Epoch 52/100
396/396 [==============================] - 0s 69us/step - loss: 1.1109 - mean_squared_error: 1.1109 - val_loss: 0.2820 - val_mean_squared_error: 0.2820
Epoch 53/100
396/396 [==============================] - 0s 47us/step - loss: 1.1093 - mean_squared_error: 1.1093 - val_loss: 0.2813 - val_mean_squared_error: 0.2813
Epoch 54/100
396/396 [==============================] - 0s 56us/step - loss: 1.1085 - mean_squared_error: 1.1085 - val_loss: 0.2844 - val_mean_squared_error: 0.2844
Epoch 55/100
396/396 [==============================] - 0s 65us/step - loss: 1.1079 - mean_squared_error: 1.1079 - val_loss: 0.2816 - val_mean_squared_error: 0.2816
Epoch 56/100
396/396 [=

Epoch 100/100
396/396 [==============================] - 0s 51us/step - loss: 1.0829 - mean_squared_error: 1.0829 - val_loss: 0.2956 - val_mean_squared_error: 0.2956


The model did much, much better this time around!

Run the cell below to get the model's predictions for both the training and validation sets. 

In [60]:
pred_train = model.predict(X_train).reshape(-1)
pred_val = model.predict(X_val).reshape(-1)

Let's look at the first 10 predictions from `pred_train`. Display those in the cell below.

In [61]:
pred_train[:10]

array([-0.18283753, -0.03986977,  0.05675085, -0.04728414, -0.1313188 ,
       -0.28910244,  0.24912313, -0.05160578,  0.13380547, -0.36836094],
      dtype=float32)

Let's manually calculate the Mean Squared Error in the cell below.  

As a refresher, here's the formula for calculating Mean Squared Error:

<img src='mse_formula.gif'>

Use `pred_train` and `Y_train` to calculate our training MSE in the cell below.  

**_HINT:_** Use numpy to make short work of this!

In [62]:
MSE_train = np.mean((pred_train-Y_train)**2)#np.mean((pred_train-Y_train)**2)
MSE_train 

1.0774242440188744

Now, calculate the MSE for our validation set in the cell below.

In [63]:
MSE_val = np.mean((pred_val-Y_val)**2)
MSE_val 

0.2955563334858838

### 2.3 Use weight initializers

Another way to increase the performance of our models is to initialize our weights in clever ways.  We'll explore some of those options below.  

#### 2.3.1  He initialization

Let's try and use a weight initializer.  We'll start with the **_He normalizer_**, which initializes the weight vector to have an average 0 and a variance of 2/n, with $n$ the number of features feeding into a layer.

In the cell below:

* Recreate the Neural Network that we created above.  This time, in the hidden layer, set the `kernel_initializer` to `"he_normal"`.
* Compile and fit the model with the same hyperparameters as we used before.  

In [64]:
model = Sequential()
model.add(Dense(8,activation='relu',kernel_initializer='he_normal',input_dim = 12))
model.add(Dense(1,activation='linear'))

model.compile(optimizer='sgd', loss='mse', metrics=['mse'])


model.fit(X_train, Y_train, batch_size=32,epochs=100,verbose=1, validation_data=(X_val, Y_val))

Train on 396 samples, validate on 99 samples
Epoch 1/100
396/396 [==============================] - 0s 1ms/step - loss: 1.5358 - mean_squared_error: 1.5358 - val_loss: 0.5187 - val_mean_squared_error: 0.5187
Epoch 2/100
396/396 [==============================] - 0s 41us/step - loss: 1.3809 - mean_squared_error: 1.3809 - val_loss: 0.4213 - val_mean_squared_error: 0.4213
Epoch 3/100
396/396 [==============================] - 0s 39us/step - loss: 1.3231 - mean_squared_error: 1.3231 - val_loss: 0.3819 - val_mean_squared_error: 0.3819
Epoch 4/100
396/396 [==============================] - 0s 41us/step - loss: 1.2950 - mean_squared_error: 1.2950 - val_loss: 0.3729 - val_mean_squared_error: 0.3729
Epoch 5/100
396/396 [==============================] - 0s 61us/step - loss: 1.2748 - mean_squared_error: 1.2748 - val_loss: 0.3596 - val_mean_squared_error: 0.3596
Epoch 6/100
396/396 [==============================] - 0s 43us/step - loss: 1.2583 - mean_squared_error: 1.2583 - val_loss: 0.3514 - val

396/396 [==============================] - 0s 54us/step - loss: 1.1334 - mean_squared_error: 1.1334 - val_loss: 0.3226 - val_mean_squared_error: 0.3226
Epoch 51/100
396/396 [==============================] - 0s 79us/step - loss: 1.1310 - mean_squared_error: 1.1310 - val_loss: 0.3175 - val_mean_squared_error: 0.3175
Epoch 52/100
396/396 [==============================] - 0s 70us/step - loss: 1.1289 - mean_squared_error: 1.1289 - val_loss: 0.3192 - val_mean_squared_error: 0.3192
Epoch 53/100
396/396 [==============================] - 0s 79us/step - loss: 1.1287 - mean_squared_error: 1.1287 - val_loss: 0.3158 - val_mean_squared_error: 0.3158
Epoch 54/100
396/396 [==============================] - 0s 73us/step - loss: 1.1290 - mean_squared_error: 1.1290 - val_loss: 0.3195 - val_mean_squared_error: 0.3195
Epoch 55/100
396/396 [==============================] - 0s 59us/step - loss: 1.1265 - mean_squared_error: 1.1265 - val_loss: 0.3149 - val_mean_squared_error: 0.3149
Epoch 56/100
396/396 [=

Epoch 100/100
396/396 [==============================] - 0s 60us/step - loss: 1.1051 - mean_squared_error: 1.1051 - val_loss: 0.3495 - val_mean_squared_error: 0.3495


Great!

Run the cells below to get training and validation predictions are recalculate our MSE for each. 

In [65]:
pred_train = model.predict(X_train).reshape(-1)
pred_val = model.predict(X_val).reshape(-1)

MSE_train = np.mean((pred_train-Y_train)**2)
MSE_val = np.mean((pred_val-Y_val)**2)

In [66]:
print(MSE_train) 
print(MSE_val) 

1.1027603217769826
0.3494797474491159


The initializer does not really help us to decrease the MSE. We know that initializers can be particularly helpful in deeper networks, and our network isn't very deep. What if we use the `Lecun` initializer with a `tanh` activation?

#### 2.3.2  Lecun initialization

In the cell below, recreate the network again.  This time, set hidden layer's activation to `'tanh'`, and the `kernel_initializer` to `'lecun_normal'`.

Then, fit and compile the model as did before.  

In [67]:
model = Sequential()
model.add(Dense(8,activation='tanh',kernel_initializer='lecun_normal',input_dim = 12))
model.add(Dense(1,activation='linear'))

model.compile(optimizer='sgd', loss='mse', metrics=['mse'])


model.fit(X_train, Y_train, batch_size=32,epochs=100,verbose=1, validation_data=(X_val, Y_val))

Train on 396 samples, validate on 99 samples
Epoch 1/100
396/396 [==============================] - 0s 1ms/step - loss: 1.2734 - mean_squared_error: 1.2734 - val_loss: 0.3582 - val_mean_squared_error: 0.3582
Epoch 2/100
396/396 [==============================] - 0s 48us/step - loss: 1.2039 - mean_squared_error: 1.2039 - val_loss: 0.3485 - val_mean_squared_error: 0.3485
Epoch 3/100
396/396 [==============================] - 0s 45us/step - loss: 1.1903 - mean_squared_error: 1.1903 - val_loss: 0.3313 - val_mean_squared_error: 0.3313
Epoch 4/100
396/396 [==============================] - 0s 75us/step - loss: 1.1781 - mean_squared_error: 1.1781 - val_loss: 0.3197 - val_mean_squared_error: 0.3197
Epoch 5/100
396/396 [==============================] - 0s 50us/step - loss: 1.1743 - mean_squared_error: 1.1743 - val_loss: 0.3187 - val_mean_squared_error: 0.3187
Epoch 6/100
396/396 [==============================] - 0s 72us/step - loss: 1.1673 - mean_squared_error: 1.1673 - val_loss: 0.3120 - val

396/396 [==============================] - 0s 49us/step - loss: 1.1278 - mean_squared_error: 1.1278 - val_loss: 0.2997 - val_mean_squared_error: 0.2997
Epoch 51/100
396/396 [==============================] - 0s 45us/step - loss: 1.1275 - mean_squared_error: 1.1275 - val_loss: 0.2981 - val_mean_squared_error: 0.2981
Epoch 52/100
396/396 [==============================] - 0s 52us/step - loss: 1.1283 - mean_squared_error: 1.1283 - val_loss: 0.2977 - val_mean_squared_error: 0.2977
Epoch 53/100
396/396 [==============================] - 0s 47us/step - loss: 1.1255 - mean_squared_error: 1.1255 - val_loss: 0.2986 - val_mean_squared_error: 0.2986
Epoch 54/100
396/396 [==============================] - 0s 48us/step - loss: 1.1261 - mean_squared_error: 1.1261 - val_loss: 0.2972 - val_mean_squared_error: 0.2972
Epoch 55/100
396/396 [==============================] - 0s 102us/step - loss: 1.1267 - mean_squared_error: 1.1267 - val_loss: 0.2956 - val_mean_squared_error: 0.2956
Epoch 56/100
396/396 [

Epoch 100/100
396/396 [==============================] - 0s 58us/step - loss: 1.1198 - mean_squared_error: 1.1198 - val_loss: 0.2997 - val_mean_squared_error: 0.2997


Now, run the cells below to get the predictions and calculate the MSE for training and validation again.  

In [68]:
pred_train = model.predict(X_train).reshape(-1)
pred_val = model.predict(X_val).reshape(-1)

MSE_train = np.mean((pred_train-Y_train)**2)
MSE_val = np.mean((pred_val-Y_val)**2)

In [69]:
print(MSE_train) 
print(MSE_val) 

1.1142488443889955
0.29972757202588085


## 3. Optimization

Another option we have is to play with the optimizers we choose for gradient descent during our back propagation step.  So far, we've only made use of basic `'sgd'`, or **_Stochastic Gradient Descent_**.  However, there are more advanced optimizers available to use will often converge to better minima, usually in a quicker fashion. 

In this lab, we'll try the two most popular methods: **_RMSprop_** and **_adam_**.

### 3.1 RMSprop

In the cell below, recreate the original network that we built in this lab--no kernel intialization parameter, and the activation set to `'relu'`. 

This time, when you compile the model, set the `optimizer` parameter to `"rmsprop"`.  No changes to the `fit()` call are needed--keep those parameters the same.  

In [70]:
model = Sequential()
model.add(Dense(8,activation='relu',input_dim = 12))
model.add(Dense(1,activation='linear'))

model.compile(optimizer='rmsprop', loss='mse', metrics=['mse'])


model.fit(X_train, Y_train, batch_size=32,epochs=100,verbose=1, validation_data=(X_val, Y_val))

Train on 396 samples, validate on 99 samples
Epoch 1/100
396/396 [==============================] - 1s 1ms/step - loss: 1.5947 - mean_squared_error: 1.5947 - val_loss: 0.7918 - val_mean_squared_error: 0.7918
Epoch 2/100
396/396 [==============================] - 0s 47us/step - loss: 1.4720 - mean_squared_error: 1.4720 - val_loss: 0.6662 - val_mean_squared_error: 0.6662
Epoch 3/100
396/396 [==============================] - 0s 49us/step - loss: 1.3991 - mean_squared_error: 1.3991 - val_loss: 0.5790 - val_mean_squared_error: 0.5790
Epoch 4/100
396/396 [==============================] - 0s 78us/step - loss: 1.3465 - mean_squared_error: 1.3465 - val_loss: 0.5030 - val_mean_squared_error: 0.5030
Epoch 5/100
396/396 [==============================] - 0s 67us/step - loss: 1.3080 - mean_squared_error: 1.3080 - val_loss: 0.4571 - val_mean_squared_error: 0.4571
Epoch 6/100
396/396 [==============================] - 0s 66us/step - loss: 1.2817 - mean_squared_error: 1.2817 - val_loss: 0.4217 - val

396/396 [==============================] - 0s 61us/step - loss: 1.1157 - mean_squared_error: 1.1157 - val_loss: 0.2975 - val_mean_squared_error: 0.2975
Epoch 51/100
396/396 [==============================] - 0s 54us/step - loss: 1.1147 - mean_squared_error: 1.1147 - val_loss: 0.3003 - val_mean_squared_error: 0.3003
Epoch 52/100
396/396 [==============================] - 0s 59us/step - loss: 1.1136 - mean_squared_error: 1.1136 - val_loss: 0.2991 - val_mean_squared_error: 0.2991
Epoch 53/100
396/396 [==============================] - 0s 46us/step - loss: 1.1120 - mean_squared_error: 1.1120 - val_loss: 0.2987 - val_mean_squared_error: 0.2987
Epoch 54/100
396/396 [==============================] - 0s 53us/step - loss: 1.1116 - mean_squared_error: 1.1116 - val_loss: 0.2988 - val_mean_squared_error: 0.2988
Epoch 55/100
396/396 [==============================] - 0s 64us/step - loss: 1.1100 - mean_squared_error: 1.1100 - val_loss: 0.2961 - val_mean_squared_error: 0.2961
Epoch 56/100
396/396 [=

396/396 [==============================] - 0s 56us/step - loss: 1.0850 - mean_squared_error: 1.0850 - val_loss: 0.2856 - val_mean_squared_error: 0.2856
Epoch 100/100
396/396 [==============================] - 0s 65us/step - loss: 1.0842 - mean_squared_error: 1.0842 - val_loss: 0.2857 - val_mean_squared_error: 0.2857


Now, run the cell below to get predictions and compute the MSE again.

In [71]:
pred_train = model.predict(X_train).reshape(-1)
pred_val = model.predict(X_val).reshape(-1)

MSE_train = np.mean((pred_train-Y_train)**2)
MSE_val = np.mean((pred_val-Y_val)**2)

In [72]:
print(MSE_train) 
print(MSE_val) 

1.0814013076818048
0.2857202546502471


### 3.2 Adam

Another popular optimizer is **_adam_**, which stands for `Adaptive Moment Estimation`. This is an optimzer that was created and open-sourced by a team at OpenAI, and is generally seen as the go-to choice for optimizers today. Adam combines the RMSprop algorithm with the concept of momentum, and is generally very effective at getting converging quickly.  

In the cell below, create the same network that we did above, but this time, set the optimizer parameter to `'adam'`.  Leave all other parameters the same. 

In [73]:
model = Sequential()
model.add(Dense(8,activation='relu',kernel_initializer='he_normal',input_dim = 12))
model.add(Dense(1,activation='linear'))

model.compile(optimizer='adam', loss='mse', metrics=['mse'])


hist = model.fit(X_train, Y_train, batch_size=32,epochs=100,verbose=1, validation_data=(X_val, Y_val))

Train on 396 samples, validate on 99 samples
Epoch 1/100
396/396 [==============================] - 1s 2ms/step - loss: 3.4673 - mean_squared_error: 3.4673 - val_loss: 2.5199 - val_mean_squared_error: 2.5199
Epoch 2/100
396/396 [==============================] - 0s 53us/step - loss: 3.0129 - mean_squared_error: 3.0129 - val_loss: 2.1414 - val_mean_squared_error: 2.1414
Epoch 3/100
396/396 [==============================] - 0s 68us/step - loss: 2.6664 - mean_squared_error: 2.6664 - val_loss: 1.8256 - val_mean_squared_error: 1.8256
Epoch 4/100
396/396 [==============================] - 0s 66us/step - loss: 2.3905 - mean_squared_error: 2.3905 - val_loss: 1.5740 - val_mean_squared_error: 1.5740
Epoch 5/100
396/396 [==============================] - 0s 82us/step - loss: 2.1643 - mean_squared_error: 2.1643 - val_loss: 1.3879 - val_mean_squared_error: 1.3879
Epoch 6/100
396/396 [==============================] - 0s 72us/step - loss: 2.0034 - mean_squared_error: 2.0034 - val_loss: 1.2344 - val

396/396 [==============================] - 0s 54us/step - loss: 1.1643 - mean_squared_error: 1.1643 - val_loss: 0.3402 - val_mean_squared_error: 0.3402
Epoch 51/100
396/396 [==============================] - 0s 49us/step - loss: 1.1631 - mean_squared_error: 1.1631 - val_loss: 0.3390 - val_mean_squared_error: 0.3390
Epoch 52/100
396/396 [==============================] - 0s 50us/step - loss: 1.1613 - mean_squared_error: 1.1613 - val_loss: 0.3361 - val_mean_squared_error: 0.3361
Epoch 53/100
396/396 [==============================] - 0s 51us/step - loss: 1.1599 - mean_squared_error: 1.1599 - val_loss: 0.3343 - val_mean_squared_error: 0.3343
Epoch 54/100
396/396 [==============================] - 0s 75us/step - loss: 1.1590 - mean_squared_error: 1.1590 - val_loss: 0.3329 - val_mean_squared_error: 0.3329
Epoch 55/100
396/396 [==============================] - 0s 91us/step - loss: 1.1576 - mean_squared_error: 1.1576 - val_loss: 0.3311 - val_mean_squared_error: 0.3311
Epoch 56/100
396/396 [=

396/396 [==============================] - 0s 68us/step - loss: 1.1215 - mean_squared_error: 1.1215 - val_loss: 0.2964 - val_mean_squared_error: 0.2964
Epoch 100/100
396/396 [==============================] - 0s 60us/step - loss: 1.1198 - mean_squared_error: 1.1198 - val_loss: 0.2993 - val_mean_squared_error: 0.2993


In [74]:
pred_train = model.predict(X_train).reshape(-1)
pred_val = model.predict(X_val).reshape(-1)

MSE_train = np.mean((pred_train-Y_train)**2)
MSE_val = np.mean((pred_val-Y_val)**2)

In [75]:
print(MSE_train) 
print(MSE_val) 

1.1198195705836131
0.2992929805638284


### 3.3 Learning rate decay with momentum


The final item that we'll get practice with in this lab is implementing a **_Learning Rate Decay_** strategy, along with **_Momentum_**.  We'll accomplish this by creating a `SGD` object and setting learning rate, decay, and momentum parameters at initialization.  In this way, we can then pass in the `SGD` object we've initialized to our specificataions during the compile step, rather than just a string representing an off-the-shelf `'SGD'`  optimizer.  

In the cell below:

* Create a `SGD` optimizer, which can be found in the `optimizers` module.  
    * Set the `lr` parameter to  `0.03`.
    * Set the `decay` parameter to `0.0001`
    * Set the `momentum` parameter to `0.9`.
* Recreate the same network we used during the previous example.  
* Set the optimizer parameter during the compile step to the `sgd` object we created. 
* Fit the model with the same hyperparameters as we used before.  

In [76]:
sgd = optimizers.SGD(lr=0.03, decay=0.0001, momentum=0.9)
model = Sequential()
model.add(Dense(8, input_dim=12, activation='relu'))
model.add(Dense(1, activation = 'linear'))

model.compile(optimizer= sgd ,loss='mse',metrics=['mse'])
hist = model.fit(X_train, Y_train, batch_size=32, epochs=100, validation_data = (X_val, Y_val), verbose = 0)

Finally, run the cell below to calcluate the MSE for our final version of this model and see how a learning rate decay strategy affected the model. 

In [77]:
pred_train = model.predict(X_train).reshape(-1)
pred_val = model.predict(X_val).reshape(-1)

MSE_train = np.mean((pred_train-Y_train)**2)
MSE_val = np.mean((pred_val-Y_val)**2)

In [78]:
print(MSE_train) 
print(MSE_val) 

0.9331872067365746
0.5253523983199277


## Further reading

https://machinelearningmastery.com/dropout-regularization-deep-learning-models-keras/

https://machinelearningmastery.com/grid-search-hyperparameters-deep-learning-models-python-keras/

https://machinelearningmastery.com/regression-tutorial-keras-deep-learning-library-python/

https://stackoverflow.com/questions/37232782/nan-loss-when-training-regression-network